In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import dask
import pop_tools
import glob
from functools import partial
from CASutils import readdata_utils as read

dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '50GB',
    processes = 1,
    queue = 'casper',
    local_directory = '$TMPDIR',
    resource_spec = 'select=1:ncpus=1:mem=20GB',
    project='P04010022',
    walltime='04:00:00',
    interface='ib0')

# scale up
cluster.scale(40)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34963 instead
  http_address["port"], self.http_server.port


In [5]:
client

Client Scheduler: tcp://10.12.206.54:38041 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/34963/status,Cluster Workers: 40 Cores: 40 Memory: 858.99 GB


In [6]:
pathout="/glade/scratch/islas/python/singleforcing/DATA_SORT/cesm1_picontrol/"

### Read in TEMP and SALT

In [7]:
topdir="/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/ocn/proc/tseries/monthly/"
filestemp = sorted(glob.glob(topdir+"/TEMP/b.e11.B1850C5CN.*.TEMP.*.nc"))
filessalt = sorted(glob.glob(topdir+"/SALT/b.e11.B1850C5CN.*.SALT.*.nc"))

In [8]:
tempz = []
saltz = []
timebnds = []
for ifile in np.arange(0,len(filestemp),1):
    print(ifile)
    temp = xr.open_dataset(filestemp[ifile], chunks={"time":300, "z_t":60, "nlat":120, "nlon":120})
    tarea = temp.TAREA
    dz = temp.dz
    z_t = temp.z_t
    timebndst = temp.time_bound

    salt = xr.open_dataset(filessalt[ifile], chunks={"time":300, "z_t":60, "nlat":120, "nlon":120})
    temp = temp.isel(z_t=slice(0,20)).TEMP
    salt = salt.isel(z_t=slice(0,20)).SALT

    dz = dz.isel(z_t=slice(0,20))
    z_t = z_t.isel(z_t=slice(0,20))
    
    tempzt = (temp*dz).sum('z_t')/dz.sum('z_t')
    saltzt = (salt*dz).sum('z_t')/dz.sum('z_t')
    
    tempzt = tempzt.load()
    saltzt = saltzt.load()
    
    tempz.append(tempzt)
    saltz.append(saltzt)
    timebnds.append(timebndst)
    
tempz = xr.concat(tempz, dim='time')
saltz = xr.concat(saltz, dim='time')
timebnds = xr.concat(timebnds, dim='time')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [9]:
tempz = tempz.load()
saltz = saltz.load()

In [10]:
tempz = tempz.rename('TEMP')
saltz = saltz.rename('SALT') 

### Fix the time

In [11]:
diff = np.array(timebnds.isel(d2=1)) - np.array(timebnds.isel(d2=0))
diff = diff/2.
newtime = np.array(timebnds.isel(d2=0)) + diff
tempz['time'] = newtime
saltz['time'] = newtime

### Area average

In [12]:
wgts = tarea
wgts = xr.where( (wgts.TLONG > 300) & (wgts.TLONG < 325), wgts, 0)
wgts = xr.where( (wgts.TLAT > 50) & (wgts.TLAT < 65), wgts, 0)

tempz_w = tempz.weighted(wgts.fillna(0))
saltz_w = saltz.weighted(wgts.fillna(0))
#pdz_w = pdz.weighted(wgts.fillna(0))
#rho_model_w = rho_modelz.weighted(wgts.fillna(0))

tempzm = tempz_w.mean(("nlon","nlat"))
saltzm = saltz_w.mean(("nlon","nlat"))
#pdzm = pdz_w.mean(("nlon","nlat"))
#rho_model_zm = rho_model_w.mean(("nlon","nlat"))

/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/dask/array/einsumfuncs.py:241: PerformanceWarning: Increasing number of chunks by factor of 12
  **kwargs
/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/dask/array/einsumfuncs.py:241: PerformanceWarning: Increasing number of chunks by factor of 12
  **kwargs


### Do the EOS calculation

In [13]:
ref_depth = xr.DataArray(np.zeros(np.shape(tempzm)), dims=tempzm.dims, coords=tempzm.coords) + 101.5
rho,drhods,drhodt = pop_tools.eos(salt=saltzm, temp=tempzm, return_coefs=True,depth=ref_depth)

In [14]:
tempz_anom = tempzm - tempzm.mean('time')
saltz_anom = saltzm - saltzm.mean('time')
#pdz_anom = pdzm - pdzm.mean('time')
#rho_model_anom = rho_model_zm - rho_model_zm.mean('time')

rho_anom_temp = tempz_anom*drhodt
rho_anom_salt = saltz_anom*drhods

In [15]:
rho_anom = rho - rho.mean('time')

In [16]:
rho_anom_salt = rho_anom_salt/1000.

In [17]:
rho_anom_temp = rho_anom_temp.rename('TEMP_part')
rho_anom_salt = rho_anom_salt.rename('SALT_part')
rho_anom = rho_anom.rename('RHO_from_EOS')

rho_anom_temp.to_netcdf("/glade/scratch/islas/python/singleforcing/DATA_SORT/cesm1_picontrol/RHO_contributions_CESM1_picontrol.nc")
rho_anom_salt.to_netcdf("/glade/scratch/islas/python/singleforcing/DATA_SORT/cesm1_picontrol/RHO_contributions_CESM1_picontrol.nc", mode="a")
rho_anom.to_netcdf("/glade/scratch/islas/python/singleforcing/DATA_SORT/cesm1_picontrol/RHO_contributions_CESM1_picontrol.nc", mode="a")

In [18]:
cluster.close()